## ResNet34 Epoch=15 learn_rate=0.005

In [28]:
import torch
import torch.nn as nn
from torchvision.models import resnet34, ResNet34_Weights
from PIL import Image
import torch.nn.functional as F
import os
from tqdm import tqdm  
from sklearn.metrics import precision_score, recall_score, accuracy_score
model=resnet34(weights=ResNet34_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(model.fc.in_features, 4)
model.load_state_dict(torch.load('./model_34_15_0.005.pth'))
#print(model.state_dict)
preprocess=ResNet34_Weights.IMAGENET1K_V1.transforms()
classes=['carton','other','politic','sex']

/tmp/ipykernel_5690/1595056301.py:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./model_34_15_0.005.pth'))


In [29]:
def get_img(path:str):
    img=Image.open(path)
    img=preprocess(img)
    return img

In [30]:
model.eval()
def predict(img):
    output=model(torch.stack([img]))
    prob=F.softmax(output,dim=1)
    ind=torch.argmax(prob)
    return classes[ind]

In [35]:
def evaluate_model(val_folder):
    y_true = []
    y_pred = []

    for label in os.listdir(val_folder):
        label_folder = os.path.join(val_folder, label)
        if os.path.isdir(label_folder):
            for img_name in tqdm(os.listdir(label_folder)):
                img_path = os.path.join(label_folder, img_name)
                img = get_img(img_path)
                pred = predict(img)

                # 将标签转为索引
                true_label_index = classes.index(label)
                y_true.append(true_label_index)
                pred_index=classes.index(pred)
                y_pred.append(pred_index)
    # 计算查准率、查全率和准确率
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    accuracy = accuracy_score(y_true, y_pred)

    for i, class_name in enumerate(classes):
        print(f"Class: {class_name}")
        print(f"  Precision: {precision[i]:.2f}")
        print(f"  Recall: {recall[i]:.2f}")

    print(f"Overall Accuracy: {accuracy:.2f}")

In [36]:
evaluate_model('./val')

100%|██████████| 40/40 [01:27<00:00,  2.19s/it]

Class: carton
  Precision: 0.95
  Recall: 0.95
Class: other
  Precision: 0.92
  Recall: 0.82
Class: politic
  Precision: 0.79
  Recall: 0.93
Class: sex
  Precision: 0.95
  Recall: 0.88
Overall Accuracy: 0.89
